# Coroutines
A line such as yield item produces a value that is received by the caller of next(…), and it also gives way, suspending the execution of the generator so that the caller may proceed until it’s ready to consume another value by invoking next() again. A coroutine is syntactically like a generator: just a function with the yield keyword in its body. However, in a coroutine, yield usually appears on the right side of an expression (e.g., datum = yield), and it may or may not produce a value—if there is no
expression after the yield keyword, the generator yields None.  

In [1]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)


In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x0000000004E9B5C8>

In [3]:
import inspect
inspect.getgeneratorstate(my_coro)

'GEN_CREATED'

In [4]:
next(my_coro)

-> coroutine started


In [5]:
inspect.getgeneratorstate(my_coro)

'GEN_SUSPENDED'

It’s crucial to understand that the execution of the coroutine is suspended exactly at the yield keyword.

In [7]:
import traceback

try:
    my_coro.send(42)
except:
    traceback.print_exc()


Traceback (most recent call last):
  File "<ipython-input-7-dc54d64efaaf>", line 4, in <module>
    my_coro.send(42)
StopIteration


In [8]:
inspect.getgeneratorstate(my_coro)

'GEN_CLOSED'

## Example: Coroutine to Compute a Running Average

In [9]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count


In [10]:
coro_avg = averager()
coro_avg

<generator object averager at 0x000000000536ABA0>

In [11]:
inspect.getgeneratorstate(coro_avg)

'GEN_CREATED'

In [12]:
next(coro_avg)

In [13]:
inspect.getgeneratorstate(coro_avg)  # hits yield

'GEN_SUSPENDED'

In [14]:
coro_avg.send(10)  # yields average and waits to receive the next term

10.0

In [15]:
inspect.getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [16]:
coro_avg.send(20)

15.0

In [17]:
coro_avg.send(30)

20.0

## Decorators for Coroutine Priming
We can't do much with coroutines without priming them. We must always remember to call next(my_coro) before my_coro.send(x). 

In [18]:
from functools import wraps

def coroutine(func):
    ''' Decorator: primes 'func' by advancing to first 'yield' '''
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    return primer


In [19]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count


In [20]:
avg = averager()
avg

<generator object averager at 0x000000000536AA40>

In [21]:
avg.send(10)

10.0

In [22]:
avg.send(20)

15.0

In [23]:
avg.send(30)

20.0

## Coroutine Termination and Exception Handling

In [24]:
avg = averager()

In [25]:
avg.send(40)

40.0

In [26]:
avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

An unhandled exception within a coroutine propagates to the caller of the next or send that triggered it. In this case avg.send().

In [27]:
avg.send(50)

StopIteration: 

A sentinel value that tells the coroutine to exit. Constant built-in singletons like None and Ellipsis are convenient sentinel values. Ellipsis has the advantage of being quite unusual in data streams. Another sentinel value I’ve seen used is StopIteration—the class itself, not an instance of it (and not raising it). In other words, using it like: my_coro.send(StopIteration).

In [28]:
class DemoException(Exception):
    ''' An exception type for the demonstration. '''

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')


In [29]:
exc_coro = demo_exc_handling()

In [31]:
next(exc_coro)

-> coroutine started


In [32]:
exc_coro.send(11)

-> coroutine received: 11


In [33]:
exc_coro.send(22)

-> coroutine received: 22


In [34]:
exc_coro.close()

In [35]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

generator.close() Causes the yield expression where the generator was paused to raise a GeneratorExit exception. No error is reported to the caller if the generator does not handle that exception or raises StopIteration—usually by running to completion.

## Returning a Value from a Coroutine

In [2]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count

    return Result(count, average)


the coroutine must terminate in order to return, hence the break if term is None.

In [3]:
coro_avg = averager()
coro_avg

<generator object averager at 0x0000000004F1F7D8>

In [4]:
next(coro_avg)

In [5]:
coro_avg.send(10)

In [6]:
coro_avg.send(20)

In [7]:
coro_avg.send(None)

StopIteration: Result(count=2, average=15.0)

Catching StopIteration lets us get the value returned by averager.

In [8]:
coro_avg = averager()

In [9]:
next(coro_avg)

In [10]:
coro_avg.send(10)

In [11]:
coro_avg.send(20)

In [12]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [13]:
result

Result(count=2, average=15.0)

We could also do...

In [15]:
numbers = [10, 20, 30]

avg = averager()  # create the generator
next(avg)         # prime
numbers.append(None)
for index, value in enumerate(numbers):
    try:
        avg.send(value)
    except StopIteration as exc:
        result = exc.value

print(result)

Result(count=3, average=20.0)


This roundabout way of getting the return value from a coroutine makes more sense when we realize it was defined as part of PEP 380, and the yield from construct handles it automatically by catching StopIteration internally. 

## Using yield from

In [18]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

list(gen())

['A', 'B', 1, 2]

Can be writtin as...

In [19]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

We can chain from iterables.

In [20]:
def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

The real nature of yield from cannot be demonstrated with simple iterables; it requires the mind-expanding use of nested generators. That’s why PEP 380, which introduced yield from, is titled 'Syntax for Delegating to a Subgenerator.'

In [21]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


# the delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# the client code, a.k.a. the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # important!
    
    # print(results) # uncomment to debug
    report(results)


    # output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit))

data = {
 'girls;kg':
     [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
 'girls;m':
     [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
 'boys;kg':
     [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
 'boys;m':
     [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

if __name__ == '__main__':
    main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


## The Meaning of yield from

In [23]:
gen1 = (x for x in "ABC")
print(gen1)
print(list(gen1))

<generator object <genexpr> at 0x0000000004F1FE08>
['A', 'B', 'C']


In [24]:
gen2 = (x for x in "DEF")
print(gen2)
print(list(gen2))

<generator object <genexpr> at 0x0000000004F1FAF0>
['D', 'E', 'F']


In [25]:
def func():
    yield from (x for x in "ABC")
    yield from (x for x in "DEF")
    
for x in func():
    print(x)

A
B
C
D
E
F


## Use Case: Coroutines for Discrete Event Simulation
A discrete event simulation (DES) is a type of simulation where a system is modeled as a sequence of events. In a DES, the simulation “clock” does not advance by fixed increments, but advances directly to the simulated time of the next modeled event.

In [1]:
import collections

Event = collections.namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    ''' Yield to simulator issuing event at each state change '''
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    
    yield Event(time, ident, 'going home')


In [2]:
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [3]:
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [4]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [5]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [6]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [7]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [8]:
from inspect import getgeneratorstate
getgeneratorstate(taxi)

'GEN_SUSPENDED'

In [10]:
import traceback

try:
    taxi.send(_.time + 10)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-10-a869b4731446>", line 4, in <module>
    taxi.send(_.time + 10)
AttributeError: 'str' object has no attribute 'time'


## Chapter Summary
There is also a an excellent tutorial on coroutines from [David Beazley](http://www.dabeaz.com/coroutines/).

***